<a href="https://colab.research.google.com/github/r-parkinson/r-parkinson/blob/main/SIT796_1_2C_v0_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI Gym**

Standard AI Gym setup as provided from Prac 1

In [1]:
# install required system dependencies
# install required system dependencies
!apt-get install -y xvfb x11-utils 
!apt-get install x11-utils > /dev/null 2>&1
!pip install PyVirtualDisplay==2.0.* \
  PyOpenGL==3.1.* \
  PyOpenGL-accelerate==3.1.* \
  gym[box2d]==0.17.* 
!pip install pyglet

Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


Import standard libraries to support gym setup

In [2]:
import gym
import numpy as np
import base64
import io
import IPython
from gym.wrappers import Monitor
from IPython import display
from pyvirtualdisplay import Display

# **Policy Formulation**

The Segway balance problem is similar to the CartPole-v1 Gym environment, although it should be noted the forward motion of the Segway, and the designed physical stability measures deviate from the cart pole enviroment, but its motor conrtol based in the inverted pendulum problem.
---

In [3]:
def pcy(obst, ts):
    # get all the variables out of observation state 
    cartpos , catvel , polang , velatip = obst

    #print(env.observation_space.high)
    #print(env.observation_space.low)
    #print (polang , velatip ," ")
    # After tyring lots ot different methods to solve this is the simplest, 
    #  policy that can be used to keep the pole balanced it seems the tip 
    #  velocity needs to be kept under control, while correcting the pole 
    #  angle 2 out of 3 steps.
     
    if ts % 3 == 0:    
      if velatip > 0: 
        pact = 1
 
      if velatip < 0:
        pact = 0
    else: 
      
      if velatip > 0: 
        if polang > 0: pact = 1
        if polang < 0: pact = 0
      else:
        if polang > 0: pact = 1 
        if polang < 0: pact = 0
    
    #print ("t",ts , "vat", velatip, "pol" ,polang,"a" ,actions)    
    return pact

Setup the enviroment Cart Pole version 1, and define the observation sapce, for the cart pole the reward is 1 if pole remains up and 0 if it falls past 12% from the centre line. The number of epocs or loops here is important as that gives an indication of how long the pole can ballance for.  After doing reaseach on the cart pole it is designed to represent the physics of a pole balancing a friction less cart, so as it complys to the law of physics it does not need a policy that is derived from RL to keep the pole upright.

In [4]:
!rm ./vid/*.* # Clean up the video before starting

#Total loops this is set to match the environment
tc = 500 


# starts up the display
dis = Display()
dis.start()

env = gym.make('CartPole-v1')
env = Monitor(env,'./vid',force=True)
#print (env.action_space)
obs = env.reset()

for ts in range(tc):
    # Get the policy from function
    act = pcy(obs, ts)  
    obs, rew, don, _ = env.step (act)  # Pass the action chosen by the policy to the environment
    
    # The reward is the number of time steps before the pole falls over so in this case 
    #print(" ")
    if don:
        if rew == 1:
          print("The pole did not fall after", ts+1 , "epochs")
        else:
          print("The pole fell after", ts+1, "epochs")
    
env.reset()
        


The pole did not fall after 500 epochs


array([-0.01375622,  0.01100001, -0.00454004,  0.02191575])

The output display is as given in the practical.

In [5]:

for f in env.videos:
    video = io.open(f[0], 'r+b').read()
    encoded = base64.b64encode(video)

    display.display(display.HTML(data="""
        <video alt="test" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
        """.format(encoded.decode('ascii'))))